In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt


In [2]:
pdbfile='ranked_0.pdb'
afmodel='result_model_3.pkl'
atoms=[2,16,26,47,57,74,98,117,127,139,146,153,167,178,189,196,220,234,248,262]


In [32]:
with open(afmodel, 'rb') as f:
  data = pickle.load(f)
bins = data['distogram']['bin_edges']
bins = np.append(bins,2*bins[-1]-bins[-2])
logits = data['distogram']['logits']
logits = np.where(logits > 50, 50, logits)
probs = np.exp(logits)/(1.0 + np.exp(logits))


In [63]:
atom_str = ','.join(map(str,atoms))
bins_str = ','.join(map(str,bins))
with open('plumed.dat','w') as p:
    p.write(f"""
ALPHA_FOLD ...
LABEL=afcv
ATOMS={atom_str}
LAMBDA=3
DISTANCES={bins_str}
""")
    for d in range(len(bins)):
        rows=[]
        for i in range(len(atoms)):
            rows.append(','.join(map(str,probs[i,:,d])))
        p.write(f"LOGIT_MATRIX{d}=%s\n" % (','.join(rows)))
    p.write("... ALPHA_FOLD\n")